<a href="https://colab.research.google.com/github/Dishaa01423/film_equipment_rental_service/blob/main/Source_code_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install langchain_groq
!pip install tiktoken
!pip install chromadb
!pip install langchain_community
!pip install langchain langchain_groq chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) .

In [2]:
import os
os.environ['GROQ_API_KEY'] = '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'

In [3]:
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
import sqlite3
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Initialize Groq LLM
groq_api_key = os.environ.get('GROQ_API_KEY')

if not groq_api_key:
    print("GROQ_API_KEY not found in environment variables.")
    groq_api_key = input("Please enter your Groq API key: ")

try:
    llm = ChatGroq(temperature=0, groq_api_key=groq_api_key)
except Exception as e:
    print(f"Error initializing ChatGroq: {e}")
    print("Please make sure you have a valid Groq API key.")
    exit(1)

# Connect to the database
conn = sqlite3.connect('equipment_rental.db')
cursor = conn.cursor()

# Email classification function
def classify_email(email_content):
    prompt = PromptTemplate(
        input_variables=["email"],
        template="Classify the following email as either 'inquiry', 'review', 'assistance', or 'other':\n\n{email}\n\nClassification:",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(email_content)

# Inquiry handling function
def handle_inquiry(email_content):
    # Extract item name from email
    item_name = extract_item_name(email_content)

    # Check database for availability
    cursor.execute("SELECT * FROM equipment WHERE name = ?", (item_name,))
    item = cursor.fetchone()

    if item:
        return f"The {item[1]} is available for rent at ${item[4]} per day."
    else:
        # Suggest similar items
        cursor.execute("SELECT name FROM equipment WHERE category = (SELECT category FROM equipment WHERE name = ?) AND availability = 1 LIMIT 3", (item_name,))
        similar_items = cursor.fetchall()
        return f"We're sorry, but the {item_name} is not available. You might be interested in: " + ", ".join([item[0] for item in similar_items])

# Review handling function
def handle_review(email_content):
    sentiment = analyze_sentiment(email_content)
    if sentiment == 'positive':
        return "Thank you for your positive review! We'd greatly appreciate it if you could share your experience on our social media pages."
    else:
        # Add to CRM for follow-up
        add_to_crm(email_content)
        return "We're sorry to hear about your experience. A customer service representative will be in touch shortly. As a token of our apology, please accept this gift voucher."

# Assistance request handling function
def handle_assistance(email_content):
    # Load and process FAQ documents
    loader = TextLoader('faq.txt')
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    # Create vector store
    embeddings = HuggingFaceEmbeddings()
    docsearch = Chroma.from_documents(texts, embeddings)

    # Search for relevant information
    docs = docsearch.similarity_search(email_content)

    if docs:
        context = docs[0].page_content
        prompt = PromptTemplate(
            input_variables=["context", "question"],
            template="Based on the following information:\n\n{context}\n\nAnswer the following question:\n\n{question}\n\nAnswer:",
        )
        chain = LLMChain(llm=llm, prompt=prompt)
        return chain.run(context=context, question=email_content)
    else:
        # Escalate to customer service
        escalate_to_customer_service(email_content)
        return "We couldn't find an immediate solution to your issue. A customer service representative will be in touch shortly to assist you."

# Email Forwarding Function
def forward_to_customer_service(email_content):
    # Email configuration
    sender_email = os.environ.get('SENDER_EMAIL')
    sender_password = os.environ.get('SENDER_PASSWORD')
    customer_service_email = "customerservice@example.com"
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    # Create the email message
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = customer_service_email
    message['Subject'] = "Customer Service: Forwarded Email"

    # Email body
    body = f"The following email has been forwarded to customer service:\n\n{email_content}"
    message.attach(MIMEText(body, 'plain'))

    try:
        # Create a secure SSL context
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()

        # Login to the email server
        server.login(sender_email, sender_password)

        # Send the email
        server.send_message(message)

        print(f"Email successfully forwarded to {customer_service_email}")
        return "Email successfully forwarded to customer service."

    except Exception as e:
        print(f"An error occurred while forwarding the email: {str(e)}")
        return "An error occurred while forwarding the email to customer service."

    finally:
        server.quit()

# Main function to process emails
def process_email(email_content):
    category = classify_email(email_content)

    if category == 'inquiry':
        return handle_inquiry(email_content)
    elif category == 'review':
        return handle_review(email_content)
    elif category == 'assistance':
        return handle_assistance(email_content)
    else:
        # Forward to customer service
        forward_to_customer_service(email_content)
        return "Your email has been forwarded to our customer service team for further assistance."